### Installing Dependencies

**Step 1: Install Dependencies**
We need to install following components to run pyspark seamlessly:
OpenJDK 8,
Spark Environment,
FindSpark package

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz
!tar xf spark-3.2.0-bin-hadoop3.2.tgz
!pip install -q findspark

**Step 2: Add environment variables**
After installing dependencies, we need to some variables to the environment so that pyspark knows where to look for using dependencies. We can do that using following commands:

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "spark-3.2.0-bin-hadoop3.2"

**Step 3: Initilize pyspark**
Finally, we just need to initilize pyspark which can be easily achieved using third-party package named findspark as shown below:

In [3]:
import findspark
findspark.init()

You can try running following commands to check if pyspark is properly installed or not:

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=pyspark-shell>

## Mounting Google Drive

In [6]:
from google.colab import drive

# Mount your Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
%cd /content/drive/MyDrive/cloud_computing/Project

/content/drive/MyDrive/cloud_computing/Project


# Cleaning dataframes

#### Reading CSV file

In [8]:
import pandas as pd

# read in CSV file with low_memory=False
df = pd.read_csv('/content/drive/MyDrive/cloud_computing/Project/apple_tweets_2022.csv', engine='python')

In [8]:
df.shape

(963789, 8)

In [9]:
df.head()

,date,rawContent,replyCount,retweetCount,likeCount,quoteCount,hashtags,cashtags
0,2022-12-30 23:59:26+00:00,Fi puked up her dinner and woke up hungry n no...,3,1,48,0,NaN,NaN
1,2022-12-30 23:58:55+00:00,🧵 My Public Records Act request to the Santa C...,3,17,44,4,NaN,NaN
2,2022-12-30 23:58:40+00:00,A space where imagination meets reality. @Sean...,2,5,10,0,NaN,NaN
3,2022-12-30 23:57:45+00:00,If I ever had to get another car without Apple...,0,0,7,0,NaN,NaN
4,2022-12-30 23:56:02+00:00,"Da big apple baby, NYC, Gotham, the only place...",0,3,30,0,NaN,NaN


In [10]:
# Checking for null values
df.isna().sum()

date                 0
rawContent           0
replyCount           0
retweetCount         0
likeCount            0
quoteCount           0
hashtags        777817
cashtags        952797
dtype: int64

#### Dropping null values

In [11]:
df.dropna(subset=['date','rawContent'],inplace = True)

In [12]:
df.shape

(963789, 8)

#### Dropping columns

In [13]:
df.drop(['hashtags', 'cashtags'], axis=1, inplace=True)

In [14]:
df.columns

Index(['date', 'rawContent', 'replyCount', 'retweetCount', 'likeCount',
       'quoteCount'],
      dtype='object')

#### Filling null values

In [15]:
df[['replyCount','retweetCount','likeCount','quoteCount']] = df[['replyCount','retweetCount','likeCount','quoteCount']].fillna(0)

In [16]:
df.isna().sum()

date            0
rawContent      0
replyCount      0
retweetCount    0
likeCount       0
quoteCount      0
dtype: int64

In [17]:
df.head()

,date,rawContent,replyCount,retweetCount,likeCount,quoteCount
0,2022-12-30 23:59:26+00:00,Fi puked up her dinner and woke up hungry n no...,3,1,48,0
1,2022-12-30 23:58:55+00:00,🧵 My Public Records Act request to the Santa C...,3,17,44,4
2,2022-12-30 23:58:40+00:00,A space where imagination meets reality. @Sean...,2,5,10,0
3,2022-12-30 23:57:45+00:00,If I ever had to get another car without Apple...,0,0,7,0
4,2022-12-30 23:56:02+00:00,"Da big apple baby, NYC, Gotham, the only place...",0,3,30,0


In [18]:
df.dtypes

date            object
rawContent      object
replyCount       int64
retweetCount     int64
likeCount        int64
quoteCount       int64
dtype: object

#### Converting column datatype

In [22]:
df['quoteCount'] = df['quoteCount'].astype('float64')
df['replyCount'] = df['replyCount'].astype('float64')
df['retweetCount'] = df['retweetCount'].astype('float64')
df['likeCount'] = df['likeCount'].astype('float64')

In [23]:
df.dtypes

date             object
rawContent       object
replyCount      float64
retweetCount    float64
likeCount       float64
quoteCount      float64
dtype: object

In [24]:
pwd

'/content/drive/MyDrive/cloud_computing/Project'

In [25]:
df.to_csv('Cleaned Data/cleaned_apple_tweets.csv',index = False)

#### Reading cleaned csv file into pyspark dataframe

In [9]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/cloud_computing/Project/Cleaned Data/cleaned_apple_tweets.csv', engine = 'python')

In [10]:
df.shape

(963789, 6)

In [11]:
# convert the Pandas DataFrame to a PySpark DataFrame
apple_twts_df = spark.createDataFrame(df)

/content/spark-3.2.0-bin-hadoop3.2/python/pyspark/sql/pandas/conversion.py:371: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [12]:
apple_twts_df.count()

963789

In [30]:
apple_twts_df.show(10)

+--------------------+--------------------+----------+------------+---------+----------+
|                date|          rawContent|replyCount|retweetCount|likeCount|quoteCount|
+--------------------+--------------------+----------+------------+---------+----------+
|2022-12-30 23:59:...|Fi puked up her d...|       3.0|         1.0|     48.0|       0.0|
|2022-12-30 23:58:...|🧵 My Public Reco...|       3.0|        17.0|     44.0|       4.0|
|2022-12-30 23:58:...|A space where ima...|       2.0|         5.0|     10.0|       0.0|
|2022-12-30 23:57:...|If I ever had to ...|       0.0|         0.0|      7.0|       0.0|
|2022-12-30 23:56:...|Da big apple baby...|       0.0|         3.0|     30.0|       0.0|
|2022-12-30 23:55:...|A green apple in ...|       2.0|         3.0|     27.0|       0.0|
|2022-12-30 23:54:...|why i just seen a...|       2.0|         0.0|      7.0|       0.0|
|2022-12-30 23:54:...|PSA: if your Appl...|       0.0|         0.0|      6.0|       0.0|
|2022-12-30 23:54:...|

### Filtering stock tweets

In [13]:
import re

def extract_stock_tweets(tweet):
    # Define a regular expression pattern to match stock market-related keywords and hashtags
    pattern = r'\b$aapl\b|\bstock market\b|\bstocks\b|\bshares\b|\btrading\b|\binvesting\b|\binvestor\b|\bbullish\b|\bbearish\b|\bportfolio\b|\bETF\b|\bindex\b|\b#stockmarket\b|\bstockmarkets\b|\bstockmarketnews\b|\bstockmarketinvesting\b|\bindianstockmarket\b|\bstockmarketindia\b|\bstockmarketeducation\b|\bstockmarketcrash\b|\bstockmarkettips\b|\bstockmarketquotes\b|\bstockmarkettrader\b|\bpakistanstockmarket\b|\bstockmarketanalysis\b|\bstockmarketlab\b|\bstockmarketinvestor\b|\bstockmarketmemes\b|\bstockmarketca\b|\bstockmarketmindgames\b|\bwoodstockmarket\b|\bstockmarketing\b|\bstockmarketmonitor\b|\bstockmarkettrading\b|\bstockmarketcourse\b|\bstockmarketupdate\b|\busstockmarket\b|\bstockmarket101\b|\bphilippinestockmarket\b|\bstockmarketopportunities\b|\binstastockmarket\b|\bstockmarketprice\b|\bstockmarketph\b|\bthestockmarket\b|\bstockmarketgame\b|\bstockmarketcrash2022\b|\bstockmarketadvisory\b|\bstockmarkettraining\b|\blearnstockmarket\b|\bstockmarketgains\b|\bstockmarketleader\b|\blivestockmarketing\b'

    if re.search(pattern, tweet, re.IGNORECASE):
        return True
    else:
        return False

In [14]:
from pyspark.sql.functions import udf
from pyspark.sql.types import BooleanType,StringType

# Convert the custom function into a PySpark UDF
custom_filter_udf = udf(extract_stock_tweets, BooleanType())

apple_stock_twts_df = apple_twts_df.filter(custom_filter_udf(apple_twts_df['rawContent']))

In [15]:
apple_stock_twts_df.show(10)

+--------------------+--------------------+----------+------------+---------+----------+
|                date|          rawContent|replyCount|retweetCount|likeCount|quoteCount|
+--------------------+--------------------+----------+------------+---------+----------+
|2022-12-30 21:34:...| Buy. Apple. shares.|       2.0|         0.0|     61.0|       0.0|
|2022-12-30 19:41:...|"GLOBAL STOCKS LO...|      11.0|        20.0|     46.0|       0.0|
|2022-12-30 19:15:...|@mareiw1ngz @diew...|       0.0|         1.0|     48.0|       0.0|
|2022-12-30 18:52:...|I've seen a lot o...|       3.0|         1.0|     16.0|       0.0|
|2022-12-30 16:09:...|@CliveRhymester @...|       1.0|         0.0|      5.0|       0.0|
|2022-12-30 14:56:...|$AAPL 1 year ago ...|       3.0|         2.0|     31.0|       0.0|
|2022-12-30 09:57:...|4. The metaverse ...|      11.0|        53.0|   2444.0|       3.0|
|2022-12-30 03:11:...|NEW PODCAST AVAIL...|       1.0|         2.0|     32.0|       0.0|
|2022-12-30 00:32:...

In [34]:
apple_stock_twts_df.count()

7525

### Preprocessing the filtered stock tweets

In [16]:
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
stopwords = stopwords.words('english') 
import nltk
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

# Define a function to perform lemmatization
def lemmatization(tweet):
    words = tweet.split()
    lem_sentence = " ".join([lemmatizer.lemmatize(word) for word in words])
    return lem_sentence

def remove_stop_words(tweet):
  words = tweet.split()  # split the sentence into words
  filtered_words = [word for word in words if word.casefold() not in stopwords]
  filtered_text = ' '.join(filtered_words)  # join the filtered words into a sentence
  return filtered_text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [17]:
!pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 13.5 MB/s eta 0:00:00


In [18]:
import re
import contractions

def preprocess_tweets(tweet):
    tweet = tweet.lower() #converting lowercase
    tweet = re.sub(r'@\w+', ' ', tweet) #removes all mentions from the tweet
    tweet = re.sub(r'#\w+', ' ', tweet) #removes all hashtags from the tweet
    tweet = re.sub(r'http\S+', ' ', tweet) #removes all URLs from the tweet
    tweet = re.sub(r'www\S+', ' ', tweet) #removes all URLs that begin with "www" from the tweet
    tweet = re.sub(r'\d+\w*|\w*\d+\w*', ' ', tweet) #replaces alphanumeric or numeric with word number
    tweet = re.sub(r'(.)\1\1+', r'\1\1', tweet) #keep two consecutive duplicates and remove the rest
    tweet = re.sub(r"'s\b", ' ',tweet) # remove 's
    tweet = re.sub(r'[^\w\s]', ' ', tweet) #removes all non-alphanumeric characters and non-whitespace characters from the tweet
    tweet = re.sub(r'[^\x00-\x7F]+', ' ', tweet)  #removes all non-ASCII characters from the tweet 
    tweet = re.sub(r'[^\w\s#@/:%.,_-]', ' ', tweet) #removes any remaining non-alphanumeric characters
    tweet = contractions.fix(tweet)
    tweet = lemmatization(tweet)
    tweet = remove_stop_words(tweet)
    return tweet

In [19]:
from pyspark.sql.functions import udf
from pyspark.sql.types import BooleanType,StringType

# Convert the preprocessing function into a PySpark UDF
custom_filter_udf = udf(preprocess_tweets, StringType())
apple_stock_twts_cleaned_df = apple_stock_twts_df.withColumn("cleanedTweets",custom_filter_udf(apple_stock_twts_df['rawContent']))

In [39]:
apple_stock_twts_cleaned_df.show(10)

+--------------------+--------------------+----------+------------+---------+----------+--------------------+
|                date|          rawContent|replyCount|retweetCount|likeCount|quoteCount|       cleanedTweets|
+--------------------+--------------------+----------+------------+---------+----------+--------------------+
|2022-12-30 21:34:...| Buy. Apple. shares.|       2.0|         0.0|     61.0|       0.0|     buy apple share|
|2022-12-30 19:41:...|"GLOBAL STOCKS LO...|      11.0|        20.0|     46.0|       0.0|global stock lose...|
|2022-12-30 19:15:...|@mareiw1ngz @diew...|       0.0|         1.0|     48.0|       0.0|sorry deleted ori...|
|2022-12-30 18:52:...|I've seen a lot o...|       3.0|         1.0|     16.0|       0.0|seen lot people g...|
|2022-12-30 16:09:...|@CliveRhymester @...|       1.0|         0.0|      5.0|       0.0|shareholder every...|
|2022-12-30 14:56:...|$AAPL 1 year ago ...|       3.0|         2.0|     31.0|       0.0|aapl year ago mar...|
|2022-12-3

### Calculating sentiment score for tweets

In [20]:
# Download the Vader lexicon if it is not already downloaded
nltk.download('vader_lexicon')

from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Initialize the sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Define a UDF to calculate sentiment score for a text
def calculate_sentiment_score(text):
    # Analyze the sentiment of the text
    scores = analyzer.polarity_scores(text)
    
    # Return the compound sentiment score
    return scores['compound']

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [21]:
# Register the UDF
sentiment_udf = udf(calculate_sentiment_score, DoubleType())

apple_stock_twts_cleaned_df = apple_stock_twts_cleaned_df.withColumn("sentimentScore",sentiment_udf(apple_stock_twts_cleaned_df['cleanedTweets']))

In [42]:
apple_stock_twts_cleaned_df.show(10)

+--------------------+--------------------+----------+------------+---------+----------+--------------------+--------------+
|                date|          rawContent|replyCount|retweetCount|likeCount|quoteCount|       cleanedTweets|sentimentScore|
+--------------------+--------------------+----------+------------+---------+----------+--------------------+--------------+
|2022-12-30 21:34:...| Buy. Apple. shares.|       2.0|         0.0|     61.0|       0.0|     buy apple share|         0.296|
|2022-12-30 19:41:...|"GLOBAL STOCKS LO...|      11.0|        20.0|     46.0|       0.0|global stock lose...|       -0.7269|
|2022-12-30 19:15:...|@mareiw1ngz @diew...|       0.0|         1.0|     48.0|       0.0|sorry deleted ori...|        0.0516|
|2022-12-30 18:52:...|I've seen a lot o...|       3.0|         1.0|     16.0|       0.0|seen lot people g...|       -0.1531|
|2022-12-30 16:09:...|@CliveRhymester @...|       1.0|         0.0|      5.0|       0.0|shareholder every...|        0.4767|


# Grouping by date

In [22]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

def extract_date(date):
  date_value = date.split()[0]
  return date_value

# Register the UDF
sentiment_udf = udf(extract_date, StringType())

apple_stock_twts_cleaned_df = apple_stock_twts_cleaned_df.withColumn("date",sentiment_udf(apple_stock_twts_cleaned_df['date']))

In [44]:
apple_stock_twts_cleaned_df.show(10)

+----------+--------------------+----------+------------+---------+----------+--------------------+--------------+
|      date|          rawContent|replyCount|retweetCount|likeCount|quoteCount|       cleanedTweets|sentimentScore|
+----------+--------------------+----------+------------+---------+----------+--------------------+--------------+
|2022-12-30| Buy. Apple. shares.|       2.0|         0.0|     61.0|       0.0|     buy apple share|         0.296|
|2022-12-30|"GLOBAL STOCKS LO...|      11.0|        20.0|     46.0|       0.0|global stock lose...|       -0.7269|
|2022-12-30|@mareiw1ngz @diew...|       0.0|         1.0|     48.0|       0.0|sorry deleted ori...|        0.0516|
|2022-12-30|I've seen a lot o...|       3.0|         1.0|     16.0|       0.0|seen lot people g...|       -0.1531|
|2022-12-30|@CliveRhymester @...|       1.0|         0.0|      5.0|       0.0|shareholder every...|        0.4767|
|2022-12-30|$AAPL 1 year ago ...|       3.0|         2.0|     31.0|       0.0|aa

In [23]:
from pyspark.sql.functions import col, count, sum, avg, expr, percentile_approx, abs

# Group the tweets by date and calculate the daywise positive score, negative score,
# number of positive tweets, and number of negative tweets

grouped_apple_stock_twts_df = apple_stock_twts_cleaned_df.groupBy("date").agg(
    (sum(expr("sentimentScore * likeCount")) / sum("likeCount")).alias("weighted_avg_sentiment_score"),
    # Calculate the median positive sentiment score
    percentile_approx(expr("CASE WHEN sentimentScore > 0 THEN sentimentScore END"), 0.5).alias("daywise_median_positive_score"),
    # Calculate the median negative sentiment score
    abs(percentile_approx(expr("CASE WHEN sentimentScore < 0 THEN sentimentScore END"), 0.5)).alias("daywise_median_negative_score"),
    sum((col("sentimentScore") > 0.0).cast("int")).alias("daywise_positive_tweet_count"),
    sum((col("sentimentScore") < 0.0).cast("int")).alias("daywise_negative_tweet_count"),
    avg(col("replyCount")).alias("daywise_avg_replyCount"),
    avg((col("retweetCount")).cast("int")).alias("daywise_avg_retweetCount"),
    avg((col("likeCount")).cast("int")).alias("daywise_avg_likeCount"),
    avg((col("quoteCount")).cast("int")).alias("daywise_avg_quoteCount")
)

In [46]:
grouped_apple_stock_twts_df.show(10)

+----------+----------------------------+-----------------------------+-----------------------------+----------------------------+----------------------------+----------------------+------------------------+---------------------+----------------------+
|      date|weighted_avg_sentiment_score|daywise_median_positive_score|daywise_median_negative_score|daywise_positive_tweet_count|daywise_negative_tweet_count|daywise_avg_replyCount|daywise_avg_retweetCount|daywise_avg_likeCount|daywise_avg_quoteCount|
+----------+----------------------------+-----------------------------+-----------------------------+----------------------------+----------------------------+----------------------+------------------------+---------------------+----------------------+
|2022-01-01|          0.5865996240601504|                         0.34|                         null|                           6|                           0|                   4.0|      13.166666666666666|                133.0|            

In [24]:
# replacing null values with 0
grouped_apple_stock_twts_df = grouped_apple_stock_twts_df.na.fill(value = 0.0)
grouped_apple_stock_twts_df.show(10)

+----------+----------------------------+-----------------------------+-----------------------------+----------------------------+----------------------------+----------------------+------------------------+---------------------+----------------------+
|      date|weighted_avg_sentiment_score|daywise_median_positive_score|daywise_median_negative_score|daywise_positive_tweet_count|daywise_negative_tweet_count|daywise_avg_replyCount|daywise_avg_retweetCount|daywise_avg_likeCount|daywise_avg_quoteCount|
+----------+----------------------------+-----------------------------+-----------------------------+----------------------------+----------------------------+----------------------+------------------------+---------------------+----------------------+
|2022-01-01|          0.5865996240601504|                         0.34|                          0.0|                           6|                           0|                   4.0|      13.166666666666666|                133.0|            

In [48]:
grouped_apple_stock_twts_df.count()

364

In [25]:
# Rounding off the values using ceil
from pyspark.sql.functions import ceil

# Assume `df` is a PySpark DataFrame with a column `col` that contains values to be rounded up
grouped_apple_stock_twts_df = grouped_apple_stock_twts_df.withColumn("daywise_avg_replyCount", ceil(grouped_apple_stock_twts_df.daywise_avg_replyCount))
grouped_apple_stock_twts_df = grouped_apple_stock_twts_df.withColumn("daywise_avg_retweetCount", ceil(grouped_apple_stock_twts_df.daywise_avg_retweetCount))
grouped_apple_stock_twts_df = grouped_apple_stock_twts_df.withColumn("daywise_avg_likeCount", ceil(grouped_apple_stock_twts_df.daywise_avg_likeCount))
grouped_apple_stock_twts_df = grouped_apple_stock_twts_df.withColumn("daywise_avg_quoteCount", ceil(grouped_apple_stock_twts_df.daywise_avg_quoteCount))

In [26]:
company_id = {'google':1,'microsoft':2, 'tesla':3, 'amazon':4, 'apple':5}

In [27]:
from pyspark.sql.functions import lit
grouped_apple_stock_twts_df = grouped_apple_stock_twts_df.withColumn('company',lit(company_id['apple']))

In [28]:
grouped_apple_stock_twts_df.show(10)

+----------+----------------------------+-----------------------------+-----------------------------+----------------------------+----------------------------+----------------------+------------------------+---------------------+----------------------+-------+
|      date|weighted_avg_sentiment_score|daywise_median_positive_score|daywise_median_negative_score|daywise_positive_tweet_count|daywise_negative_tweet_count|daywise_avg_replyCount|daywise_avg_retweetCount|daywise_avg_likeCount|daywise_avg_quoteCount|company|
+----------+----------------------------+-----------------------------+-----------------------------+----------------------------+----------------------------+----------------------+------------------------+---------------------+----------------------+-------+
|2022-01-01|          0.5865996240601504|                         0.34|                          0.0|                           6|                           0|                     4|                      14|          

## Saving the file

In [29]:
# converting pyspark dataframe to pandas dataframe
DF = grouped_apple_stock_twts_df.toPandas()

DF.to_csv('Preprocessed Data/apple_preprocessed.csv', index = False)

**Note**: Similarly preprocessing is done for other companies like AMAZON, GOOGLE, TESLA, MICROSOFT, APPLE